In [2]:
# Using H2o AutoML to predict house prices

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn import metrics
from scipy import stats

from copy import deepcopy

import h2o

ModuleNotFoundError: No module named 'plotly'

## 1.Data Clean
### (1) 导入并观察数据

In [ ]:
train_df = pd.read_csv('/kaggle/input/california-house-prices/train.csv')
test_df = pd.read_csv('/kaggle/input/california-house-prices/test.csv')
train_df.shape, test_df.shape

- 根据以下散点图，删除离群点

In [ ]:
train_df=train_df.drop([3674,6055,32867,34876,43398,44091,44633])

In [ ]:
data = pd.concat([train_df['Sold Price'], train_df['Listed Price']], axis=1)
fig = px.scatter(data, x='Listed Price', y='Sold Price')
fig.show()

In [ ]:
data = pd.concat([train_df['Sold Price'], train_df['Tax assessed value']], axis=1)
fig = px.scatter(data, x='Tax assessed value', y='Sold Price')
fig.show()

In [ ]:
data = pd.concat([train_df['Sold Price'], train_df['Annual tax amount']], axis=1)
fig = px.scatter(data, x='Annual tax amount', y='Sold Price')
fig.show()

In [ ]:
data = pd.concat([train_df['Sold Price'], train_df['Last Sold Price']], axis=1)
fig = px.scatter(data, x='Last Sold Price', y='Sold Price')
fig.show()

In [ ]:
# Split features and labels
y = train_df['Sold Price'].reset_index(drop=True)
train_features = train_df.drop('Sold Price', axis=1)
test_features = test_df.copy()

features = pd.concat([train_features, test_features]).reset_index(drop=True)
features.shape

- 初步处理数据  zip列应作为字符串 并对bedroom列作简单处理

In [ ]:
def proc_bedroom(x):
    if not pd.isna(x) and not x.isdigit():
        temp = x.split(',')
        n = len(x.split(','))
        if 'Walk-in Closet' in temp:
            n -= 1
        return n
    else:
        return x

features['Bedrooms']=features['Bedrooms'].apply(lambda x: proc_bedroom(x))
features['Bedrooms'] = pd.to_numeric(features['Bedrooms'])
features['Zip'] = features['Zip'].astype('str')

- 缺省值填充

In [ ]:
#missing data
total = features.isnull().sum().sort_values(ascending=False)
percent = ((features.isnull().sum() / features.isnull().count()) * 100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data = missing_data.reset_index()
missing_data.columns = ['Name', 'Total', 'Percent']
missing_data[:10]

In [ ]:
def handle_missing(features):
    zero_fill=['Last Sold Price','Lot','Full bathrooms','Annual tax amount','Tax assessed value','Bathrooms',
               'Bedrooms','Total interior livable area','Total spaces','Garage spaces']
    none_fill=['Last Sold On','Middle School','Appliances included','Flooring','Laundry features','Cooling features',
               'Cooling','Heating features','Heating','Elementary School','High School','Parking features','Parking','Summary']
    max_fill=['Middle School Score','Middle School Distance','Elementary School Score','Elementary School Distance',
              'High School Score','High School Distance']
    mode_fill=['Year built','Region']
    for c in zero_fill:
        features[c]=features[c].fillna(0)
    for c in max_fill:
        features[c]=features[c].fillna(features[c].max())
    for c in none_fill:
        features[c]=features[c].fillna('None')
    for c in mode_fill:
        features[c]=features[c].fillna(features[c].mode()[0])
    return features

In [ ]:
features = handle_missing(features)
features.shape

- 数据转换

In [ ]:
# I want to thanks @masumrumi for sharing this amazing plot!
def plotting_3_chart(df, feature):
    ## Importing seaborn, matplotlab and scipy modules. 
    import seaborn as sns
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    from scipy import stats
    import matplotlib.style as style
    style.use('fivethirtyeight')

    ## Creating a customized chart. and giving in figsize and everything. 
    fig = plt.figure(constrained_layout=True, figsize=(12,8))
    ## creating a grid of 3 cols and 3 rows. 
    grid = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)
    #gs = fig3.add_gridspec(3, 3)

    ## Customizing the histogram grid. 
    ax1 = fig.add_subplot(grid[0, :2])
    ## Set the title. 
    ax1.set_title('Histogram')
    ## plot the histogram. 
    sns.distplot(df.loc[:,feature], norm_hist=True, ax = ax1)

    # customizing the QQ_plot. 
    ax2 = fig.add_subplot(grid[1, :2])
    ## Set the title. 
    ax2.set_title('QQ_plot')
    ## Plotting the QQ_Plot. 
    stats.probplot(df.loc[:,feature], plot = ax2)

    ## Customizing the Box Plot. 
    ax3 = fig.add_subplot(grid[:, 2])
    ## Set title. 
    ax3.set_title('Box Plot')
    ## Plotting the box plot. 
    sns.boxplot(df.loc[:,feature], orient='v', ax = ax3 );

In [ ]:
# Fix the target variable
y = np.log1p(y)

plotting_3_chart(pd.DataFrame(y), 'Sold Price')

In [ ]:
numerical_columns = features.select_dtypes(include=['int64','float64']).columns

skewed_features = features[numerical_columns].apply(lambda x: stats.skew(x)).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew value' :skewed_features})
skewness.head(20)

In [ ]:
def fix_skew(features):
    """
    This function takes in a dataframe and return fixed skewed dataframe
    """
    ## Import necessary modules 
    from scipy.special import boxcox1p
    from scipy.stats import boxcox_normmax
    
    ## Getting all the data that are not of "object" type. 
    numerical_columns = features.select_dtypes(include=['int64','float64']).columns

    # Check the skew of all numerical features
    skewed_features = features[numerical_columns].apply(lambda x: stats.skew(x)).sort_values(ascending=False)
    high_skew = skewed_features[abs(skewed_features) > 0.5]
    skewed_features = high_skew.index

    # Perform the Box-Cox transformation
    for column in skewed_features:
        features[column] = boxcox1p(features[column], boxcox_normmax(features[column] + 1))
        
    return features

In [ ]:
def reset_zero(x):
    return max(x,0)
features['Garage spaces']=features['Garage spaces'].apply(lambda x: reset_zero(x))
features['Total spaces']=features['Total spaces'].apply(lambda x: reset_zero(x))
features = fix_skew(features)
features.head()

- 将features分离  根据Variable Importances进行挑选

In [ ]:
x = features.iloc[:len(y), :]
x_test = features.iloc[len(y):, :]
x.shape, y.shape, x_test.shape

In [ ]:
selected=['Listed Price','Tax assessed value','Annual tax amount','Listed On','Elementary School Distance','Last Sold On',
'Zip','Total interior livable area','Last Sold Price','Lot','Year built','Bathrooms','High School Distance',
'Elementary School Score','Full bathrooms','Middle School Distance','Heating features','Bedrooms',
'Elementary School','Laundry features','Region','Middle School Score','Type',
'Total spaces','High School Score','Parking']

x=x[selected]
x_test=x_test[selected+['Id']]
x.head()

## 2.train

In [ ]:
import h2o

h2o.init()
hf = h2o.H2OFrame(pd.concat([x, y], axis=1))
x_test_hf = h2o.H2OFrame(x_test)

In [ ]:
predictors = hf.drop('Sold Price').columns
response = 'Sold Price'

In [ ]:
from h2o.automl import H2OAutoML

# stopping_metric: Specify the metric to use for early stopping. 
aml = H2OAutoML(
    max_models=50,
    include_algos=["XGBoost"],
    max_runtime_secs=7200,
    stopping_metric='RMSLE',
    sort_metric='RMSLE'
)

In [ ]:
# Train the model
aml.train(x=predictors,y=response,training_frame=hf)

In [ ]:
lb = aml.leaderboard; lb

## 3.predict

In [ ]:
aml.leader

发现过拟合现象比较严重，这里考虑使用前k个模型的输出取均值进行整合

Reported on train data.

**RMSLE: 0.00745891154333034**

Reported on cross-validation data.

**RMSLE: 0.013097170958722777**

In [ ]:
submission_results = pd.read_csv("/kaggle/input/california-house-prices/sample_submission.csv")
def top_k_avg_predict(k,leaderboard):
    lb=leaderboard.as_data_frame()
    ans=submission_results.iloc[:, 1]
    for i in range(k):
        model=lb.loc[i]['model_id']
        pred=h2o.get_model(model).predict(x_test_hf)
        pred=pred.as_data_frame()
        ans+=np.expm1(pred['predict'])/k
    return ans

submission_results.iloc[:, 1]=top_k_avg_predict(8,aml.leaderboard)
submission_results.head()
submission_results.to_csv('submission.csv', index=False)

In [ ]:
submission_results.head()